# Preprocessing

In [1]:
# math and data packages
import pandas as pd
import numpy as np
import math

# charting and graphics
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator

# os and file types
import os
import sys
import datetime as dt
import json
import csv
from collections import defaultdict

# images and display
import base64, io, IPython
from PIL import Image as PILImage
from IPython.display import Markdown as md
from IPython.display import display, Math, Latex



## Import data

Fix any known formatting problems here

In [2]:
# The below code fixes errors and standardizes the survey, map, and reference data sets.
# species and name slugs are created, excess columns removed, etc.


# function to make the species slugs
def to_slug(x):
    try: 
        int_data = x.split()
        data = int_data[:2]
        data = "-".join(data)
        data = data.lower()
    except:
        data = "none"
    return data

# removes suffixes from string data
def asplit(x):
    akey=x
    int_data = x.strip().split("-")
    return int_data, akey

def check_length(x):
    data = len(x[0])
    return data, x[0], x[1]

def new_data(data, suffixes, threecharacters, twocharacters):
    """Removes the specified suffix from a string"""
    
    if data[0] == 1:        
        if data[1][0].endswith(suffixes):
            new_x = data[1][:-4]
        elif data[1][0].endswith(threecharacters):
            new_x = data[1][:-3]
        else:
            new_x = data[1][0]
    else:
        
        if data[1][1] in suffixes:
            new_x = data[1][0]
        elif data[1][1].endswith(threecharacters):
            new_x = data[1][0]
        elif data[1][1].endswith(twocharacters):
            new_x = data[1][0]
        else:
            new_x = "-".join(data[1])            
    
    return {data[2]:new_x}

def change_a_column_value(df,coltocopy, coltorecieve, this_function=to_slug):
    copyname = f"{coltocopy}_copy"
    df[copyname] = df[coltocopy]
    df[coltorecieve] = df[copyname].map(lambda x: this_function(x))
    return df

def columns_to_lower(df, oldnames):
    new_names = {x:x.lower() for x in oldnames}
    return df.rename(columns=new_names, inplace=True)

# this function makes a place slug for the location names.
def to_place_slug(x):
    try: 
        int_data = x.strip("-")
        datas = int_data.split()
        data = "-".join(datas)
        data = data.lower()
    except:
        data = "none"
    return data


# load the files 

# the 2014 list of invasive species on the watch list, black list and ODE lists.
invasives = pd.read_csv("resources/inprocess/invasives.csv")

# the list of priority vascular plants in switzerland
priority = pd.read_csv("resources/inprocess/priority.csv")

# the 2018 regional redlist data on the conservation status of species provided by Infoflora
redlist = pd.read_csv("resources/inprocess/redlist.csv")

# 2020 and 2021 plant survey data
surveys = pd.read_csv("resources/inprocess/combined_survey_data.csv")

# plants from infoflora database of la chaux-de-fonds
cdf5x5 = pd.read_csv("resources/atlas5x5/Atlas5x5_550_215.csv", encoding = "utf-16")

# plants from the 4 5x5 km observation squares provided by info flora 
biel51 = pd.read_csv("resources/atlas5x5/Atlas5x5_580_220.csv", encoding = "utf-16")
biel52 = pd.read_csv("resources/atlas5x5/Atlas5x5_580_215.csv", encoding = "utf-16")
biel53 = pd.read_csv("resources/atlas5x5/Atlas5x5_580_220.csv", encoding = "utf-16")
biel54 = pd.read_csv("resources/atlas5x5/Atlas5x5_580_220.csv", encoding = "utf-16")

# !! if this is a dictionary why not use .JSON ?
# dictionary to convert 2020 survey names with 2021 survey names
sk = pd.read_csv("resources/2020-2021-keys.csv")

#locations files with additional geographic data attached
locs = pd.read_csv("resources/inprocess/surveys-2021.csv", encoding = "utf-8")
locs20 = pd.read_csv("resources/map-keys.csv", encoding = "utf-8")


# cleaning the imported files
invasives = change_a_column_value(invasives, "Latin", "species", to_slug)[["species", "list_2014", "Ordonnonce sur la dissémination des organismes (ODE)"]]
invcolumns = ["species", "list_2014", "Ordonnonce sur la dissémination des organismes (ODE)", "acopy"]
invasives = invasives.rename(columns={invcolumns[2]:"ode"})
invasives["ode"] = invasives.ode.where(invasives.ode == "ODE", "X")
invasives.drop_duplicates(subset="species", inplace=True)
invdata = invasives[["species", "list_2014","ode"]].set_index("species", drop=True)

# cleaning the imported files continued
priority = change_a_column_value(priority, "species", "species", to_slug)[["species", "Jura", "Plateau", "responsabilite", "priorite"]]
pricolumns = ["species", "Jura", "Plateau", "responsabilite", "priorite", "acopy"]
new_names = {x:x.lower() for x in pricolumns}
priority.rename(columns=new_names, inplace=True)
priority.rename(columns={"jura":"JU", "plateau":"MP"},inplace=True)
priority.drop_duplicates(subset="species", inplace=True)
priority.set_index("species", inplace=True, drop=True)

# cleaning the imported files continued
redlist = change_a_column_value(redlist, "species", "species", to_slug)[["species", "CH", "JU", "MP", "FAMILY"]]
redlist.drop_duplicates(subset="species", inplace=True)
redlist.set_index("species", inplace=True, drop=True)

# cleaning the imported files continued
cdf5x5["cdf"] = "cdf"
cdf5x5 = change_a_column_value(cdf5x5, "species", "species", to_slug)
cdf5x5.set_index("species", inplace=True, drop=True)

# cleaning the imported files continued
biel51 = change_a_column_value(biel51, "species", "species", to_slug)
biel51["biel"] = "biel"
biel51.set_index("species", inplace=True, drop=True)
biel52 = change_a_column_value(biel52, "species", "species", to_slug)
biel52["biel"] = "biel"
biel52.set_index("species", inplace=True, drop=True)
biel53 = change_a_column_value(biel53, "species", "species", to_slug)
biel53["biel"] = "biel"
biel53.set_index("species", inplace=True, drop=True)
biel54 = change_a_column_value(biel54, "species", "species", to_slug)
biel54["biel"] = "biel"
biel54.set_index("species", inplace=True, drop=True)

# cleaning the imported files continued
locs["place"]= locs.name.map(lambda x: to_place_slug(x))
locs20["place"]= locs20.place.map(lambda x: to_place_slug(x))
locs.set_index("place", inplace=True, drop = True)

surveys["places"] = surveys.place.map(lambda x:to_place_slug(x))


FileNotFoundError: [Errno 2] File b'resources/inprocess/invasives.csv' does not exist: b'resources/inprocess/invasives.csv'

In [ ]:
# Replace old 2020 place names with the new overlapping 2021 scheme. Around 50. 
# Around 50 remain in old nomenclature.
sk["y1"] = sk["2020"]
sk["y2"] = sk["2021"]
sk.set_index("2020", inplace=True, drop=True)
for a in sk.index:
    sk["y1"] = sk["y1"].map(lambda x: to_place_slug(x))
    sk["y2"] = sk["y2"].map(lambda x: to_place_slug(x))
sk.set_index("y1", inplace=True, drop=True)
sk = sk.drop(labels = ["2021"], axis = 1)
def replace_y1y2(x):
        if x in sk.index:
            data=sk.at[x,"y2"]
        else:
            data = x
        return data 

surveys['places1'] = surveys.places.map(lambda x: replace_y1y2(x))

In [4]:
speclist = ["tulipa-aximensis", "tulipa-raddii","conopodium-majus","centaurea-nigra"]

genuslist = ["salix", "ligustrum", "taraxacum", "rubus", "thymus","crepis","cotoneaster","solidago", "acer",
            "artemisia", "rosa", "hieracium","cerastium","iris", "myosotis","prunus","tulipa", "convolvulus",
            "ranunculus", "rumex", "sonchus","fragaria", "ajuga", "sanguisorba","lotus","chaerophyllum","lactuca",
            "lonicera","euphorbia","lepidium","papaver","sedum", "sambuscus","campanula","calendula","silene",
            "solanum","aquilegia","brassica","veronica","heracleum","hypochaeris","anthemis","potentilla",
            "hypericum","atriplex","geranium","leontodon","galeopsis",'epilobium', 'trifolium', 'knautia',
            'centranthus', 'chenopodium','juncus', 'arctium','geum', 'alchemilla','euphrasia', 'vicia', 'polygonum',
            'melilotus', 'carduus','teucrium', 'oenothera', 'prunella','ligustrum', 'mentha', 'lathyrus', 'parthenocissus',
             'crataegus', 'leucanthemum','bryonia', 'verbascum','delphinium','fraxinus', 'jasminum','valeriana',"plantago"]

notindiglist = ['veronica-persica', 'mentha-spicata', 'silene-dichotoma',
       'verbena-bonariensis', 'brassica-napus', 'lavandula-angustifolia',
       'centranthus-ruber', 'parthenocissus-quinquefolia',
       'scandosorbus-intermedia', 'alcea-rosea', 'oxalis-dillenii',
       'conzya-canadensis', 'persicaria-longiseta', 'oenothera-biennis',
       'anemone-hupehensis', 'packera-aurea', 'cosmos-bipinnatus',
       'onobrychis-viciifolia', 'hieracium aurantiacum',
       'ligustrum-sinense', 'viburnum-rhytidophyllum',
       'stachys-byzantina', 'eupatorium-serotinum', 'lepidium-virginicum',
       'cotoneaster-horizontalis', 'rudbeckia-hirta', 'antirrhinum-majus',
       'Salix-myrsinifolia', 'Polygonum-lapathifolium', 'phlox-pilosa',
       'rapistrum-rugosum', 'viburnum-farreri', 'quercus-suber',
       'narcissus-jonquilla', 'forsythia-xintermedia',
       'myosotis-macrophylla', 'mahonia-aquifolium', 'berberis-julianae',
       'veronica-filiformis', 'hyacinthus-orientalis',
       'narcissus-tazetta', 'fritillaria-imperialis',
       'thuja-occidentalis', 'spiraea-trilobata', 'prunus-cerasifera',
       'syringa-vulgaris', 'ribes-rubrum', 'narcissus-bulbocodium',
       'prunus-serrulata', 'prunus-cerasus', 'magnolia-liliiflora',
       'lonicera-nitida', 'berberis-thunbergii', 'acer-negundo',
       'prunus-persica', 'kerria-japonica', 'anemone-blanda',
       'jasminum-nudiflorum', 'lunaria-annua', 'malus-pumila',
       'aesculus-hippocastanum', 'veronica-chaemaedrys',
       'hyacinthoides-non-scripta', 'spiraea-chamaedryfolia',
       'corylus-maxima', 'iberis-sempervirens', 'lonicera-tatarica',
       'lonicera-morrowii', 'trifolium-resupinatum',
       'lepidium-densiflorum', 'lonicera-pileata',
       'parthenocissus-triscupidata', 'brunnera-macrophylla',
       'oxalis-stricta', 'ranunculus-verna', 'chaenomeles-speciosa',
       'trifolium-hybridum', 'aubrieta-deltoidia', 'lepidium-sativum',
       'kolkwitzia-amabilis', 'plantago-lanceolate', 'malus',
       'prunus-subhirtella', 'lonicera-fragrantissima',
       'borago-officinalis', 'impatiens-parviflora', 'erigeron annuus',
       'oenothera-parviflora', 'perovskia-atriplicifolia',
       'epilobium-ciliatum', 'sambucus', 'symphoricarpos-orbicularis',
       'dianthus-barbatus', 'fagopyrum-esculentum', 'nigella-damascena',
       'phacelia-tanacetifolia', 'matricaria-discoidea',
       'amaranthus-retroflexus', 'galium', 'spiraea-japonica',
       'conyza-canadensis', 'parthenocissus-tricuspidata',
       'ipomoea-purpurea', 'potentilla-fruticosa', 'hesperis-matronalis',
       'galinsoga-quadriradiata', 'echium-italicum']

In [5]:
sl = pd.DataFrame(speclist, columns=["species"])
sl.set_index("species", inplace=True, drop=True)
gl = pd.DataFrame(genuslist, columns = ["genus"])
gl.set_index("genus", inplace=True, drop=True)
ni = pd.DataFrame(notindiglist, columns = ["species"])
ni.set_index("species", inplace=True, drop=True)

In [6]:
# set the species identified in the 10X10 square around biel made up of the 4 5x5
# observation boxes.
biel_species = set([*biel51.index, *biel52.index, *biel53.index, *biel54.index])
bs = pd.DataFrame(index=biel_species)
bs["biel"] = "biel"

In [7]:
# Using the cleaned data sets above, the below code attaches key attributes from the following
# reference data sets: vascular plants on Switzerland's list of priority species, 2019 Regional Redlist
# and the list of invasive species.

priority = priority[["responsabilite", "priorite"]].copy()
all_species = set([*invdata.index, *priority.index, *redlist.index, *sl.index, *gl.index, *ni.index])
fd = pd.DataFrame(index=all_species)

rchmap = redlist.CH
rjumap = redlist.JU
rmpmap = redlist.MP
rfam = redlist.FAMILY

# attaches the redlist atrributes, the status at the national level, in Jura and in the Central Plateau
for a in all_species:
    if a in redlist.index:
        fd.loc[a, "rch" ] = rchmap.loc[a]
        fd.loc[a, "rmp" ] = rjumap.loc[a]
        fd.loc[a, "rju" ] = rjumap.loc[a]
        fd.loc[a,"fam"] = rfam.loc[a]

res = priority.responsabilite
pri = priority.priorite
    
# attaches the priority list attributes, the national priority level and the national responsability level.
# there is no regional breakdown provided so national figures are used in all cases
for a in all_species:
    # national priority list:
    if a in priority.index:
        fd.loc[a, "res"] = res.loc[a]
        fd.loc[a,"pri"] = pri.loc[a]

inv = invdata.list_2014
ode = invdata.ode

# attaches the invasive species attributes, namely if they have an ODE designation and if they are on the
# Black list (BL) or Watch list (WL)
for a in all_species:
    # invasive list:
    if a in invdata.index:
        fd.loc[a, "inv"] = inv.loc[a]
        fd.loc[a,"ode"] = ode.loc[a]        

for a in all_species:
    if a in speclist:
        fd.loc[a,"ns"] = "ns"

for a in all_species:
    if a in genuslist:
        fd.loc[a,"ongen"] = "ongen"
        
for a in all_species:
    if a in notindiglist:
        fd.loc[a, "ni"] = "ni"
        
for a in all_species:
    if a in cdf5x5.index:
        fd.loc[a, "cdf"] = "cdf"

for a in all_species:
    if a in bs.index:
        fd.loc[a, "biel"] = "biel"
        
        
# fills empty values with the value 'X'        
fd.fillna('X',inplace = True)

In [8]:
# prepares the biogeographical data to be attached to the survey data

locs.fillna('X',inplace = True)
locstp= locs.drop(columns = ["x","y","name","description"])
locstp20 = locs20.drop(columns = ["lon","lat"])
locstp20.set_index("place", inplace=True, drop = True)
locstp20.fillna("X",inplace = True)
locstp20["forest"] = "X"

In [9]:
# attaches the biogeographical data to the surveys through the location keys.

all_locs = set([*locstp.index, *locstp20.index])

for x in surveys.places1.unique():
    if x in all_locs:
        if x in locstp.index:
            data = locstp.loc[x]
            newcols = data.index
            for y in newcols:
                surveys.loc[surveys.places1==x, y] = data.loc[y]
        else:
            data = locstp20.loc[x]
            newcols = data.index
            for y in newcols:
                surveys.loc[surveys.places1==x, y] = data.loc[y]
    else:
        pass

In [10]:
# dictionary of misspellings and name discrepancies to be corrected in the 2021 data.

replacedict = {"hedera": "hedera-helix",
    "plantago-lanceolote": "plantago-lanceolata",
    "silene-latifolia": "silene-pratensis",
    "clinopodium-acinos": "acinos-arvensis", 
    "rubus-rubus":"rubus",
    "tripleurospermum-maritimum": "tripleurospermum-inodorum",
    "persicaria-maculosa": "polygonum-persicaria",
    "pilosella-aurantiaca": "hieracium aurantiacum",
    "chaenorhinum-minus": "chaenorrhinum-minus",
    "lactuca-muralis": "mycelis-muralis",
    "clinopodium-nepeta":"calamintha-nepeta",
    "lysimachia-arvensis": "anagallis-arvensis",
    "hylotelephium-telephium": "sedum-telephium",
    "salix-nigra":"Salix-myrsinifolia",
    "persicaria-lapathifolia": "Polygonum-lapathifolium",
    "hieracium-morulum": "hieracium-murorum",
    "pseudofumaria-lutea": "corydalis-lutea",
    "aria-edulis":"sorbus-aria",
    "narcissus-jonquillium": "narcissus-jonquilla",
    "corydalus-cava": "corydalis-cava",
    "forsythia": "forsythia-xintermedia",
    "narcissus-pseuonarcissus": "narcissus-pseudonarcissus",
    "brunnera-macrophylla": "myosotis-macrophylla",
    "berberis-aquifolium": "mahonia-aquifolium",
    "medicato-lupulina": "medicago-lupulina",
    "veronica-filliformis":"veronica-filiformis",
    "ribres-rubrum":"ribes-rubrum",
    "narcissus":"narcissus-pseudonarcissus",
    "euphorbia-helioscopa":"euphorbia-helioscopia",
    "sinapsis-arvensis": "sinapis-arvensis",
    "salix-unknown":"salix",
    "salix-cinera": "salix-cinerea",
    "malus-domestica":"malus-pumila",
    "veronica-chaemaedry":"veronica-chamaedrys",
    "buddleja-daviddii":"buddleja-davidii",
    "brunnera-macrophyla":"brunnera-macrophylla",
    "gernium-molle":"geranium-molle",
    "taraxacum-officnale":"taraxacum-officinale",
    "polygonatum-biflorum":"polygonatum-multiflorum",
    "achillea-millefolilum":"achillea-millefolium",
    "sorbus-mougeotli":"sorbus-mougeotii",
    "crataegus-germanica":"mespilus-germanica",
    "ficaria-verna":"ranunculus-verna",
    "symphtum-officinale":"symphytum-officinale",
    "crocus-vernus":"crocus-albiflorus",
    "senecio-jacobea":"senecio-jacobaea",
    "berberis-julianea":"berberis-julianae",
    "eupatorium-cannabinium":"eupatorium-cannabinum",
    "sambucus-mahaleb":"prunus-mahaleb",
    "hipppocrepis-emerus":"hippocrepis-emerus",
    "cardamine-heptahpylla":"cardamine-heptaphylla",
    "helleborous-foetidus":"helleborus-foetidus",
    "chrysoplenium-alternifolium":"chrysosplenium-alternifolium",
    "anemone-hepatica":"hepatica-nobilis",
    "primula-acaule":"primula-acaulis",
    "lavendula-angustifolia":"lavandula-angustifolia",
    "ranunclus-acris":"ranunculus-acris",
    "erigeron":"erigeron annuus",
}

# list of obersvation keys that need to be deleted.
deletelist = ["plant","hedgerow-plants"] # deleted
surveys = surveys[surveys.species != "plant"]
surveys = surveys[surveys.species != "hedgerow-plants"]

def replace_this(x,a_dict):
    if x in a_dict.keys():
        data=a_dict[x]
    else:
        data = x
    return data

surveys["species"] = surveys.species.map(lambda x: replace_this(x, replacedict))

In [11]:
# for each species slug in the survey data, the below loop attaches the redlist, priority list and
# invasives list data.
for val in surveys.species.unique():
    if val in fd.index:
        data = fd.loc[val]
        newcols = data.index
        for col in newcols:
            surveys.loc[surveys.species==val, col] = data.loc[col]            
    else:
        pass

In [12]:
#number of unique values in species column
b = surveys.species.nunique()
#genus only identification
c = surveys[surveys["ongen"]== "ongen"]
#not indigenous, not invasive (yet)
d = surveys[surveys["ni"]=="ni"]

#not indigenous, invasive
e = surveys[surveys["inv"]=="WL"]
f = surveys[surveys["inv"]=="BL"]
g = surveys[surveys["ode"]=="ode"]

h = c.species.nunique()
i = d.species.nunique()
j = e.species.nunique()
k = f.species.nunique()
l = g.species.nunique()

# number of unique native species identified :)
a = b-h-i-j-k-l

In [13]:
surveys["locname"] = surveys["places1"]
surveys = surveys.drop(columns = ["place","loc_date","places","places1"])

In [14]:
surveys.head()

,date,species,year,biogeo,canton,river,lot,forest,project,city,...,res,pri,inv,ode,ns,ongen,ni,cdf,biel,locname
0,2020-09-02,plantago-lanceolata,1,mp,be,madretschkanal,X,X,X,biel,...,X,X,X,X,X,X,X,cdf,biel,alleestrasse-madretschkanal-1
1,2020-09-02,centaurea-nigra,1,mp,be,madretschkanal,X,X,X,biel,...,X,X,X,X,ns,X,X,X,X,alleestrasse-madretschkanal-1
2,2020-09-02,plantago-media,1,mp,be,madretschkanal,X,X,X,biel,...,X,X,X,X,X,X,X,cdf,biel,alleestrasse-madretschkanal-1
3,2020-09-02,chenopodium-album,1,mp,be,madretschkanal,X,X,X,biel,...,X,X,X,X,X,X,X,cdf,biel,alleestrasse-madretschkanal-1
4,2020-09-02,centaurea-jacea,1,mp,be,madretschkanal,X,X,X,biel,...,2.0,3.0,X,X,X,X,X,cdf,biel,alleestrasse-madretschkanal-1


In [15]:
surveys.to_csv("resources/inprocess/master.csv")

### Given the list of invasive, threatened and priority species answer the following questions in relation to the survey results from 2020 and 2021

__The big numbers__

1. Number of samples
2. Number of locations
3. Date range
4. Number of species on the lists identified

### 
